In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

playground_series_s5e6_path = kagglehub.competition_download('playground-series-s5e6')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# ==============================================================================
# 1. IMPORTAÇÃO DAS BIBLIOTECAS NECESSÁRIAS
# ==============================================================================
# Importando as ferramentas essenciais para manipulação de dados,
# modelagem e visualização.

import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.metrics import top_k_accuracy_score # Usaremos para validação local
import warnings

# Ignorar warnings para manter a saída limpa
warnings.filterwarnings('ignore')

print("Bibliotecas importadas com sucesso!")

# ==============================================================================
# 2. CARREGAMENTO DOS DADOS (VERSÃO CORRIGIDA)
# ==============================================================================
# A competição nos incentiva a usar o dataset original. Vamos fazer isso!
# Carregaremos os dados sintéticos da competição e o dataset real,
# e depois os combinaremos para ter um conjunto de treinamento mais robusto.

BASE_PATH = '/kaggle/input/playground-series-s5e6/'
try:
    original_df = pd.read_csv('/kaggle/input/fertilizer-prediction/Fertilizer Prediction.csv')
    # Renomeando colunas do dataset original para bater com o da competição
    original_df.rename(columns={
        'Temparature': 'Temperature', # Corrigindo erro de digitação
        'Humidity ': 'Humidity',
        'Moisture': 'Moisture',
        'Soil Type': 'Soil_Type',
        'Crop Type': 'Crop_Type',
        'Nitrogen': 'Nitrogen',
        'Potassium': 'Potassium',
        'Phosphorous': 'Phosphorous',
        'Fertilizer Name': 'Fertilizer_Name'
    }, inplace=True)
    print("Dataset original carregado com sucesso.")
    USE_ORIGINAL_DATA = True
except FileNotFoundError:
    print("Dataset original não encontrado. Prosseguindo apenas com os dados da competição.")
    USE_ORIGINAL_DATA = False


# Carregando os dados da competição
train_df = pd.read_csv(BASE_PATH + 'train.csv')
test_df = pd.read_csv(BASE_PATH + 'test.csv')
sample_submission_df = pd.read_csv(BASE_PATH + 'sample_submission.csv')

# ==============================================================================
# CORREÇÃO: Padronizando o nome da coluna 'Temparature' imediatamente
# ==============================================================================
# Esta é a correção chave para o KeyError que você encontrou.
rename_dict = {'Temparature': 'Temperature'}
train_df.rename(columns=rename_dict, inplace=True)
test_df.rename(columns=rename_dict, inplace=True)
print("\nColuna 'Temparature' corrigida para 'Temperature' nos dataframes da competição.")
# ==============================================================================

# Guardando os IDs do teste para a submissão final
test_ids = test_df['id']
# Removendo a coluna 'id' para que os dataframes de treino e teste tenham as mesmas colunas
train_df = train_df.drop('id', axis=1)
test_df = test_df.drop('id', axis=1)

print("\nDados da competição carregados:")
print(f"Treino: {train_df.shape}")
print(f"Teste: {test_df.shape}")


# ==============================================================================
# 3. PRÉ-PROCESSAMENTO E ENGENHARIA DE FEATURES (VERSÃO CORRIGIDA)
# ==============================================================================
# Esta etapa agora funcionará sem erros, pois o nome da coluna já foi corrigido.

# --- 3.1. Limpeza e Unificação dos Nomes das Colunas ---
# Padronizar os nomes das colunas (trocar espaços por _) ajuda a evitar erros.
def clean_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        # Garante que não haja espaços no início/fim e substitui espaços internos
        clean_col = col.strip().replace(' ', '_')
        new_cols.append(clean_col)
    df.columns = new_cols
    return df

train_df = clean_col_names(train_df)
test_df = clean_col_names(test_df)
if USE_ORIGINAL_DATA:
    original_df = clean_col_names(original_df)


# --- 3.2. Combinando com o Dataset Original (se disponível) ---
if USE_ORIGINAL_DATA:
    # Unindo o dataset original ao de treino
    train_df = pd.concat([train_df, original_df[train_df.columns]], ignore_index=True)
    print(f"\nFormato do treino após combinar com dados originais: {train_df.shape}")


# --- 3.3. Engenharia de Features ("A Parte Legal") ---
# A função agora encontrará a coluna 'Temperature' sem problemas.
print("\nIniciando Engenharia de Features...")

def create_features(df):
    df_copy = df.copy()
    epsilon = 1e-6

    # Relações entre nutrientes (N-P-K)
    df_copy['N_P_Ratio'] = df_copy['Nitrogen'] / (df_copy['Phosphorous'] + epsilon)
    df_copy['N_K_Ratio'] = df_copy['Nitrogen'] / (df_copy['Potassium'] + epsilon)
    df_copy['P_K_Ratio'] = df_copy['Phosphorous'] / (df_copy['Potassium'] + epsilon)
    df_copy['N_P_K_Sum'] = df_copy['Nitrogen'] + df_copy['Phosphorous'] + df_copy['Potassium']

    # Interação entre clima e umidade do solo
    # AGORA ISSO FUNCIONARÁ
    df_copy['Temp_Humidity_Interaction'] = df_copy['Temperature'] * df_copy['Humidity']
    df_copy['Temp_Moisture_Interaction'] = df_copy['Temperature'] * df_copy['Moisture']

    return df_copy

train_df = create_features(train_df)
test_df = create_features(test_df)
print(train_df.info())
print(test_df.info())

print("Novas features criadas com sucesso!")
print("Novas colunas:", [col for col in train_df.columns if col not in test_df.columns and col != 'Fertilizer_Name'])


# --- 3.4. Codificação da Variável Alvo (Target Encoding) ---
target_encoder = LabelEncoder()
train_df['Fertilizer_Name_Encoded'] = target_encoder.fit_transform(train_df['Fertilizer_Name'])
fertilizer_classes = target_encoder.classes_
print(f"\nEncontrados {len(fertilizer_classes)} tipos de fertilizantes.")


# --- 3.5. Codificação das Features Categóricas (One-Hot Encoding) ---
categorical_features = ['Soil_Type', 'Crop_Type']
combined_df = pd.concat([train_df.drop(['Fertilizer_Name', 'Fertilizer_Name_Encoded'], axis=1), test_df], ignore_index=True)
combined_df = pd.get_dummies(combined_df, columns=categorical_features, drop_first=True)

X = combined_df[:len(train_df)]
X_test = combined_df[len(train_df):]
y = train_df['Fertilizer_Name_Encoded']

print("\nPré-processamento finalizado.")
print(f"Formato final do dataset de treino (features): {X.shape}")
print(f"Formato final do dataset de teste (features): {X_test.shape}")

In [ ]:
train_df

In [ ]:
# ==============================================================================
# 4. TREINAMENTO DO MODELO (LightGBM)
# ==============================================================================
# LightGBM é uma excelente escolha para este tipo de problema.
# Vamos treiná-lo com todos os dados de treino disponíveis.

print("\nIniciando o treinamento do modelo LightGBM...")

# Configuração dos parâmetros do LightGBM
# Estes são parâmetros iniciais sólidos. Para melhores resultados,
# eles podem ser otimizados (tuning) usando técnicas como GridSearch ou Optuna.
lgb_params = {
    'objective': 'multiclass',  # Nosso problema é de classificação com múltiplas classes
    'metric': 'multi_logloss', # Métrica de avaliação durante o treino
    'num_class': len(fertilizer_classes), # Número de classes que o modelo deve prever
    'n_estimators': 2000,       # Número de árvores. Um valor alto, mas usaremos early stopping.
    'learning_rate': 0.02,      # Taxa de aprendizado, controla o passo de cada iteração.
    'feature_fraction': 0.8,    # Usa 80% das features em cada árvore
    'bagging_fraction': 0.8,    # Usa 80% dos dados em cada árvore
    'bagging_freq': 1,
    'lambda_l1': 0.1,           # Regularização L1
    'lambda_l2': 0.1,           # Regularização L2
    'num_leaves': 31,           # Número de folhas por árvore
    'verbose': -1,              # Suprime mensagens de log do LightGBM
    'n_jobs': -1,               # Usa todos os cores de CPU disponíveis
    'seed': 42,                 # Semente para reprodutibilidade
    'boosting_type': 'gbdt',
}

# Criando o classificador
model = lgb.LGBMClassifier(**lgb_params)

# Treinando o modelo
# Aqui, não estamos usando um conjunto de validação separado porque vamos treinar
# com todos os dados disponíveis para maximizar a performance na submissão.
# Em um projeto real, usaríamos validação cruzada para ter uma estimativa mais
# robusta da performance do modelo.
model.fit(X, y)

print("Modelo treinado com sucesso!")


# ==============================================================================
# 5. PREDIÇÃO E GERAÇÃO DO ARQUIVO DE SUBMISSÃO
# ==============================================================================
# Agora, usamos nosso modelo treinado para prever no conjunto de teste.
# A métrica é MAP@3, então precisamos das 3 predições mais prováveis.

print("\nIniciando predições no conjunto de teste...")

# `predict_proba` retorna a probabilidade para cada uma das classes
probabilities = model.predict_proba(X_test)

# `argsort` nos dá os índices das classes, ordenados da menor para a maior probabilidade.
# Usamos [:, ::-1] para inverter a ordem (maior para menor).
# Pegamos os 3 primeiros com [:, :3].
top_3_preds_indices = np.argsort(probabilities, axis=1)[:, ::-1][:, :3]

# Agora, convertemos esses índices de volta para os nomes dos fertilizantes
top_3_preds_labels = target_encoder.inverse_transform(top_3_preds_indices.flatten()).reshape(top_3_preds_indices.shape)

# Formatando para o arquivo de submissão
# A competição exige que as 3 predições estejam na mesma célula, separadas por espaço.
predictions_formatted = [' '.join(preds) for preds in top_3_preds_labels]

print("Predições formatadas com sucesso.")


# --- 5.1. Criação do arquivo de submissão ---
submission_df = pd.DataFrame({
    'id': test_ids,
    'Fertilizer Name': predictions_formatted
})

# Salvando o arquivo
submission_df.to_csv('submission.csv', index=False)

print("\nArquivo de submissão 'submission.csv' criado com sucesso!")
print("Primeiras 5 linhas da submissão:")
print(submission_df.head())